<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：220px; height：150px; vertical-align：middle;">
            <img src = "../ assets/aaa.png" width = "220" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">自動トレーダー</h2>
            <span style = "color：＃ff7800;"> MCPサーバーのツールとリソースを搭載した自律エージェントを説明するエクイティ取引シミュレーション。
            </span>
        </td>
    </tr>
</table>

###週6日4日

そして今 - キャップストーンプロジェクトの紹介：


＃自律トレーダー

4人のトレーダーと1人の研究者がいるエクイティトレーディングシミュレーションは、ツールとリソースを備えた多数のMCPサーバーを搭載しています。

1.自家製アカウントMCPサーバー（エンジニアリングチームによって書かれています！）
2.フェッチ（ローカルヘッドレスブラウザを介してWebページを取得）
3。メモリ
4。勇敢な検索
5。財務データ

トレーダーのアカウントとその投資戦略に関する情報を読むためのリソース。

今日のラボの目標は、新しいPythonモジュール「Traders.py」を作成することです。

ラボで実験して探索し、準備ができたらPythonモジュールに移行します。


<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/stop.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">もう1回 -  </h2>
            <Span style = "color：＃ff7800;">実際の取引決定にこれを使用しないでください!!
            </span>
        </td>
    </tr>
</table>

In [ ]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

### トレーダーのためにMCPパラメーションを集めることから始めましょう

In [ ]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### そして今、私たちの研究者のために

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### 次に、それぞれにMcPserverstDioを作成します

In [13]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### それでは、市場調査を行うための研究者エージェントを作ってみましょう

そして、それをツールに変えてください - これがOpenai Agents SDKでどのように機能するか、そしてハンドオフとの違いを覚えていますか？

In [14]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [15]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [ ]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



### トレースを見てください

https://platform.openai.com/traces

In [ ]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

### そして今 - トレーダーエージェントを作成する

In [26]:
agent_name = "Ed"

# MCPサーバーを使用してリソースを読み取ります
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [ ]:
print(instructions)

### そして、私たちのトレーダーを運営します

In [ ]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

###その後、トレースを見てください

http://platform.openai.com/traces


In [ ]:
# そして、取引の結果を見てみましょう

await read_accounts_resource(agent_name)

###今度は、これから作られたPythonモジュールを確認する時が来ました。

`mcp_params.py`は、MCPサーバーが指定されている場所です。おなじみの友達を連れてきたことに気付くでしょう：メモリとプッシュ通知！

`templates.py`は、指示とメッセージがセットアップされる場所です（つまり、システムのプロンプトとユーザープロンプト）

`Traders.py`はそれをすべて一緒にもたらします。

あなたは私がこのようなコードで少し派手なことをしたことに気付くでしょう：

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

これは、「コンテキストマネージャーと呼ばれる」ステートメントと「」ステートメントを組み合わせるための整然とした方法であるため、このようなことをする必要はありません。

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

しかし、それは同等です。


In [2]:
from traders import Trader


In [3]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### 今、トレースを見てください

https://platform.openai.com/traces

###合計でいくつのツールを使用しましたか？

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")